In [1]:
import requests
import pandas as pd
import re
from datetime import datetime
import os
from dotenv import load_dotenv
# Load environment variables from a .env file
load_dotenv()

True

### Fetch the top 5000 delegates data 

In [2]:
def fetch_top_5000_delegates():
    """
    Fetches the top 5000 delegates using the skip parameter.
    Converts the block timestamp and latest balance to readable formats.
    Includes a 3-second timeout for API requests.
    
    Returns:
        pd.DataFrame: A DataFrame containing delegate data.
        filename
    """
    # API endpoint
    url = os.getenv('SUBGRAPH_ENDPOINT')

    # GraphQL query template with a placeholder for skip value
    query_template = """
    query MyQuery {{
      delegates(orderBy: latestBalance, orderDirection: desc, first: 1000, skip: {skip_value}) {{
        id
        latestBalance
      }}
    }}
    """

    def fetch_data(skip_value):
        """
        Executes the GraphQL query with the given skip value and returns the fetched data.
        Includes a 3-second timeout.
        """
        query = query_template.format(skip_value=skip_value)
        try:
            response = requests.post(url, json={"query": query}, timeout=5)  # Added 3-second timeout
            response.raise_for_status()  # Raise an error for HTTP issues
        except requests.Timeout:
            print(f"Request timed out for skip={skip_value}")
            raise
        except requests.RequestException as e:
            print(f"Request failed for skip={skip_value}: {e}")
            raise

        result = response.json()

        # Handle GraphQL errors
        if "errors" in result:
            print("GraphQL Error:", result["errors"])
            raise ValueError("GraphQL query failed.")

        return result.get("data", {}).get("delegates", [])

    # Initialize variables for pagination
    all_data = []  # To store all results
    skip = 0       # Start with skip=0
    batch_size = 1000

    while len(all_data) < 5000:
        try:
            # Fetch data for the current skip value
            data = fetch_data(skip)
        except (requests.Timeout, requests.RequestException) as e:
            print(f"Error fetching data for skip={skip}: {e}")
            break
        except Exception as e:
            print(f"Unexpected error for skip={skip}: {e}")
            break

        if not data:  # Exit if no more data is returned
            break

        all_data.extend(data)

        # Increment skip by batch size
        skip += batch_size

        # Stop if we've fetched 5000 records
        if len(all_data) >= 5000:
            all_data = all_data[:5000]  # Trim excess records
            break

    if all_data:
        # Convert the data to a DataFrame
        df = pd.DataFrame(all_data)

        # Rename columns to match the required names
        df.rename(columns={"id": "delegate", "latestBalance": "voting_power"}, inplace=True)

        # Convert latestBalance by dividing by 10^18
        df['voting_power'] = df["voting_power"].astype(float) / 10**18
        
        # Get the current date in the format %Y-%m-%d
        current_date = datetime.now().strftime("%Y-%m-%d")

        # Save the dataframe to CSV with the current date as the filename
        filepath = f"./Data/{current_date}.csv"
        df.to_csv(filepath, index=False)

        # Extract the filename from the path
        filename = os.path.basename(filepath)

        return df, filename
    else:
        print("No data fetched. Returning an empty DataFrame.")
        return pd.DataFrame(), None

### Distribute the Power of ACC 

In [3]:
def distribute_ACC_Power(delegates, filename):
    """
    Process a single delegate file by distributing voting power and removing the ACC delegate.

    Parameters:
    - delegate_file_path (str): Path to the delegate data CSV file.
    """
    acc_file_path = "./Data/Anticapture_Commission.csv"

    acc_delegate_address = "0x3eee61b92c36e97be6319bf9096a1ac3c04a1466"

    # Load the ACC data
    acc_data = pd.read_csv(acc_file_path, encoding='latin1')

    # Extract the file date from the filename
    file_date = datetime.strptime(os.path.basename(filename)[:-4], '%Y-%m-%d')

    # Ensure dates in ACC data are in datetime format
    acc_data.loc[:, 'start_date'] = pd.to_datetime(acc_data['start_date'], format='%Y-%m-%d')
    acc_data.loc[:, 'end_date'] = pd.to_datetime(acc_data['end_date'], format='%Y-%m-%d')

    # Check if ACC delegate address is present in the delegates data
    acc_delegate_row = delegates[delegates['delegate'].str.lower() == acc_delegate_address.lower()]
    if acc_delegate_row.empty:
        print("ACC delegate not found in the delegates data. Returning unmodified dataframe.")
        return delegates

    # Filter ACC data by season
    acc_season5 = acc_data[acc_data['season'] == 5]
    acc_season6 = acc_data[acc_data['season'] == 6]
    acc_season7 = acc_data[acc_data['season'] == 7]

    # Determine the active season based on the file date
    if (file_date >= acc_season5['start_date'].iloc[0]) and (file_date <= acc_season5['end_date'].iloc[0]):
        active_season = acc_season5
    elif (file_date >= acc_season6['start_date'].iloc[0]) and (file_date <= acc_season6['end_date'].iloc[0]):
        active_season = acc_season6
    elif (file_date >= acc_season7['start_date'].iloc[0]) and (file_date <= acc_season7['end_date'].iloc[0]):
        active_season = acc_season7
    else:
        print(f"File date {file_date} does not fall within any season. Returning unmodified dataframe.")
        return delegates

    # Standardize address formats to lowercase
    active_season['address'] = active_season['address'].str.lower()
    delegates['delegate'] = delegates['delegate'].str.lower()

    # Identify missing members from the active season and add them to the delegates dataframe
    missing_members = active_season[~active_season['address'].isin(delegates['delegate'])]
    missing_members_to_add = missing_members[['address']].rename(columns={'address': 'delegate'})
    missing_members_to_add['voting_power'] = 0
    updated_delegates = pd.concat([delegates, missing_members_to_add], ignore_index=True)

    # Get the ACC delegate's voting power
    acc_delegate_voting_power = acc_delegate_row['voting_power'].iloc[0]

    # Distribute the ACC delegate's voting power among active season members
    total_members = len(active_season)
    voting_power_per_member = acc_delegate_voting_power / total_members
    updated_delegates.loc[updated_delegates['delegate'].isin(active_season['address']),
                          'voting_power'] += voting_power_per_member

    # Remove the ACC delegate from the dataset
    updated_delegates = updated_delegates[updated_delegates['delegate'] != acc_delegate_address.lower()]

    # Sort the dataframe by voting power in descending order
    sorted_data = updated_delegates.sort_values(by='voting_power', ascending=False).reset_index(drop=True)
  
    return sorted_data

### Add missing members in the delegate list

In [4]:
# Helper function to add missing delegates
def add_missing_delegates(data, new_addresses):
    missing_addresses = set(new_addresses['address'].str.lower()) - set(data['delegate'].str.lower())
    missing_df = pd.DataFrame({
        'delegate': list(missing_addresses),
        'voting_power': 0
    })
    data = pd.concat([data, missing_df], ignore_index=True)
    return data

### Convert Dates to Datetime Format

In [5]:
def convert_dates(df):
    """
    Converts the 'start_date' and 'end_date' columns to datetime format for a given dataframe.

    Parameters:
        df (pd.DataFrame): The dataframe containing 'start_date' and 'end_date' columns.

    Returns:
        pd.DataFrame: The dataframe with 'start_date' and 'end_date' converted to datetime format.
    """
    if 'start_date' in df.columns and 'end_date' in df.columns:
        try:
            df.loc[:, 'start_date'] = pd.to_datetime(df['start_date'], dayfirst=False)
            df.loc[:, 'end_date'] = pd.to_datetime(df['end_date'], dayfirst=False)
        except Exception as e:
            print(f"Error converting dates: {e}")
    else:
        print("Warning: 'start_date' or 'end_date' columns are missing in the dataframe.")
    return df


### Add Membership of Members

In [6]:
def add_membership_columns(data, councils, file_date):
    for col_name, council_data in councils:
        # Check if council is active during the given date
        is_active = (file_date >= council_data['start_date'].iloc[0]) & (file_date <= council_data['end_date'].iloc[0])
        if is_active:
            # Add membership column for active councils
            data[col_name] = data['delegate'].apply(
                lambda x: 1 if x.lower() in council_data['address'].str.lower().values else 0   
            )
        else:
            # Default to 0 if not active 
            data[col_name] = 0
    return data

### Assign Voting Power

In [7]:
def assign_voting_power(data):
    
    # Calculate and assign voting power percentages based on active membership
    sum_vp = data[data['voting_power'] > 1]['voting_power'].sum()
    data['th_vp'] = data.apply(lambda row: (row['voting_power'] * 100) / sum_vp if row['voting_power'] > 1 else 0, axis=1)

    # Define the councils and committees along with their membership columns
    councils_and_committees = [
        ('ch_member_r2', 'ch_vp_r2'),
        ('ch_member_r3', 'ch_vp_r3'),
        ('ch_member_r4', 'ch_vp_r4'),
        ('ch_member_r5', 'ch_vp_r5'),
        ('ch_member_r6', 'ch_vp_r6'),
        ('gc_member_s3', 'gc_vp_s3'),
        ('gc_member_s4', 'gc_vp_s4'),
        ('gc_member_s5', 'gc_vp_s5'),
        ('gc_member_mm_s5', 'gc_vp_mm_s5'),
        ('dab_member_s5', 'dab_vp_s5'),
        ('coc_member_s5', 'coc_vp_s5'),
        ('gc_member_s6', 'gc_vp_s6'),
        ('gc_member_mm_s6', 'gc_vp_mm_s6'),
        ('dab_member_s6', 'dab_vp_s6'),
        ('coc_member_s6', 'coc_vp_s6'),
        ('gc_member_s7', 'gc_vp_s7'),
        ('gc_member_op_s7','gc_vp_op_s7'),
        ('dab_member_s7','dab_vp_s7'),
        ('mmc_member_s7','mmc_vp_s7')
    ]

    # Loop through each council/committee to calculate the voting power percentage
    for member_col, vp_col in councils_and_committees:
        count_member = data[member_col].sum() 
        data[vp_col] = data.apply(
            lambda row: (row[member_col] * 100) / count_member if row[member_col] == 1 else 0, axis=1
        )

    return data 

### Create Datasheet

In [8]:
import os
from datetime import datetime
import pandas as pd
import pymongo
from pymongo import MongoClient, UpdateOne
from dotenv import load_dotenv

load_dotenv()

def create_data_sheet(data, filename):
    MONGO_URI = os.getenv('MONGODB_URI')
    DATABASE_NAME = 'CPI'
    COLLECTION_NAME = 'delegate_data'

    try:
        client = MongoClient(MONGO_URI)
        db = client[DATABASE_NAME]
        collection = db[COLLECTION_NAME]

        new_version = int(datetime.utcnow().timestamp())
        file_date = datetime.strptime(filename[:-4], '%Y-%m-%d')
        
        # Load data from other CSV files
        citizens_round2 = pd.read_csv("./Data/Round 2.csv", encoding='latin1')
        citizens_round3 = pd.read_csv("./Data/Round 3.csv", encoding='latin1')
        citizens_round4 = pd.read_csv("./Data/Round 4.csv", encoding='latin1')
        citizens_round5 = pd.read_csv("./Data/Round 5.csv", encoding='latin1')
        citizens_round6 = pd.read_csv("./Data/Round 6.csv", encoding='latin1')
        grants = pd.read_csv("./Data/Grants_Council.csv", encoding='latin1')
        grants_mm = pd.read_csv("./Data/Grants_Council_MM.csv", encoding='latin1')
        grants_op = pd.read_csv("./Data/Grants_Council_Operations.csv", encoding='latin1')
        dab = pd.read_csv("./Data/Developer_Advisory_Board.csv", encoding='latin1')
        coc = pd.read_csv("./Data/Code_of_Conduct_Council.csv", encoding='latin1')
        mmc = pd.read_csv("./Data/Milestone_and_Metrics_Council.csv", encoding='latin1')
        

        # Drop rows with null values in the 'address' column for each DataFrame
        citizens_round2.dropna(subset=['address'], inplace=True)
        citizens_round3.dropna(subset=['address'], inplace=True)
        citizens_round4.dropna(subset=['address'], inplace=True)
        citizens_round5.dropna(subset=['address'], inplace=True)
        citizens_round6.dropna(subset=['address'], inplace=True)

        grants.dropna(subset=['address'], inplace=True)
        grants_mm.dropna(subset=['address'], inplace=True)
        grants_op.dropna(subset=['address'], inplace=True)
        dab.dropna(subset=['address'], inplace=True)
        coc.dropna(subset=['address'], inplace=True)
        mmc.dropna(subset=['address'], inplace=True)

        # Filter grants data by season 
        grants_season3 = grants[grants['season'] == 3]

        grants_season4 = grants[grants['season'] == 4]

        grants_season5 = grants[grants['season'] == 5]
        grants_mm_season5 = grants_mm[grants_mm['season'] == 5]
        dab_season5 = dab[dab["season"] == 5]
        coc_season5 = coc[coc["season"] == 5]

        grants_season6 = grants[grants['season'] == 6]
        grants_mm_season6 = grants_mm[grants_mm['season'] == 6]
        dab_season6 = dab[dab["season"] == 6]
        coc_season6 = coc[coc["season"] == 6]

        grants_season7 = grants[grants['season'] == 7]
        grants_op_season7 = grants_op[grants_op['season'] == 7]
        dab_season7 = dab[dab["season"] == 7]
        mmc_season7 = mmc[mmc["season"] == 7]

        # Calling the function and capturing the returned data
        citizens_round2 = convert_dates(citizens_round2)
        citizens_round3 = convert_dates(citizens_round3)
        citizens_round4 = convert_dates(citizens_round4)
        citizens_round5 = convert_dates(citizens_round5)
        citizens_round6 = convert_dates(citizens_round6)
        grants_season3 = convert_dates(grants_season3)
        grants_season4 = convert_dates(grants_season4)
        grants_season5 = convert_dates(grants_season5)
        grants_mm_season5 = convert_dates(grants_mm_season5)
        dab_season5 = convert_dates(dab_season5)
        coc_season5 = convert_dates(coc_season5)
        grants_season6 = convert_dates(grants_season6)
        grants_mm_season6 = convert_dates(grants_mm_season6)
        dab_season6 = convert_dates(dab_season6)
        coc_season6 = convert_dates(coc_season6)

        grants_season7 = convert_dates(grants_season7)
        grants_op_season7 = convert_dates(grants_op_season7)
        dab_season7 = convert_dates(dab_season7)
        mmc_season7 = convert_dates(mmc_season7)

        # Add missing delegates from various rounds and councils
        data = add_missing_delegates(data, citizens_round2)
        data = add_missing_delegates(data, citizens_round3)
        data = add_missing_delegates(data, citizens_round4)
        data = add_missing_delegates(data, citizens_round5)
        data = add_missing_delegates(data, citizens_round6)
        data = add_missing_delegates(data, grants_season3)
        data = add_missing_delegates(data, grants_season4)
        data = add_missing_delegates(data, grants_season5)
        data = add_missing_delegates(data, grants_mm_season5)
        data = add_missing_delegates(data, dab_season5)
        data = add_missing_delegates(data, coc_season5)
        data = add_missing_delegates(data, grants_season6)
        data = add_missing_delegates(data, grants_mm_season6)
        data = add_missing_delegates(data, dab_season6)
        data = add_missing_delegates(data, coc_season6)
        data = add_missing_delegates(data, grants_season7)
        data = add_missing_delegates(data, grants_op_season7)
        data = add_missing_delegates(data, dab_season7)
        data = add_missing_delegates(data, mmc_season7)
        
        # Add the columns 
        data['th_vp'] = None
        data['ch_member_r2'] = None
        data['ch_vp_r2'] = None
        data['ch_member_r3'] = None
        data['ch_vp_r3'] = None
        data['ch_member_r4'] = None
        data['ch_vp_r4'] = None
        data['ch_member_r5'] = None
        data['ch_vp_r5'] = None
        data['ch_member_r6'] = None
        data['ch_vp_r6'] = None

        data['gc_member_s3'] = None
        data['gc_vp_s3'] = None
        data['gc_member_s4'] = None
        data['gc_vp_s4'] = None

        data['gc_member_s5'] = None
        data['gc_vp_s5'] = None
        data['gc_member_mm_s5'] = None
        data['gc_vp_mm_s5'] = None
        data['sc_member_s5'] = None
        data['sc_vp_s5'] = None
        data['coc_member_s5'] = None
        data['coc_vp_s5'] = None
        data['dab_member_s5'] = None
        data['dab_vp_s5'] = None

        data['gc_member_s6'] = None
        data['gc_vp_s6'] = None
        data['gc_member_mm_s6'] = None
        data['gc_vp_mm_s6'] = None
        data['sc_member_s6'] = None
        data['sc_vp_s6'] = None
        data['coc_member_s6'] = None
        data['coc_vp_s6'] = None
        data['dab_member_s6'] = None
        data['dab_vp_s6'] = None

        data['gc_member_s7'] = None
        data['gc_vp_s7'] = None
        data['gc_member_op_s7'] = None
        data['gc_vp_op_s7'] = None
        data['sc_member_s7'] = None
        data['sc_vp_s7'] = None
        data['dab_member_s7'] = None
        data['dab_vp_s7'] = None
        data['mmc_member_s7'] = None
        data['mmc_vp_s7'] = None

        councils = [
        ('ch_member_r2', citizens_round2),
        ('ch_member_r3', citizens_round3),
        ('ch_member_r4', citizens_round4),
        ('ch_member_r5', citizens_round5),
        ('ch_member_r6', citizens_round6),
        ('gc_member_s3', grants_season3),
        ('gc_member_s4', grants_season4),
        ('gc_member_s5', grants_season5),
        ('gc_member_mm_s5', grants_mm_season5),
        ('dab_member_s5', dab_season5),
        ('coc_member_s5', coc_season5),
        ('gc_member_s6', grants_season6),
        ('gc_member_mm_s6', grants_mm_season6),
        ('dab_member_s6', dab_season6),
        ('coc_member_s6', coc_season6),
        ('gc_member_s7', grants_season7),
        ('gc_member_op_s7', grants_op_season7),
        ('dab_member_s7', dab_season7),
        ('mmc_member_s7', mmc_season7)
    ]

        # Add membership columns dynamically
        data = add_membership_columns(data, councils, file_date)
        print("Membership columns added successfully!",data)

        data = assign_voting_power(data)

        # Fill all null values in the dataframe with 0
        data.fillna(0, inplace=True)
        # Create bulk operations
        bulk_operations = []
        current_time = datetime.utcnow()

        for _, row in data.iterrows():
            voting_power = {
                'vp': float(row['voting_power']) if 'voting_power' in row else 0,
                'th_vp': float(row['th_vp']) if 'th_vp' in row else 0
            }
            
            vp_columns = [
                'ch_vp_r2', 'ch_vp_r3', 'ch_vp_r4', 'ch_vp_r5', 'ch_vp_r6',
                'gc_vp_s3', 'gc_vp_s4', 'gc_vp_s5', 'gc_vp_mm_s5',
                'gc_vp_s6', 'gc_vp_mm_s6', 'sc_vp_s5', 'sc_vp_s6',
                'coc_vp_s5', 'coc_vp_s6', 'dab_vp_s5', 'dab_vp_s6', 'gc_vp_s7',
                'gc_vp_op_s7','dab_vp_s7','mmc_vp_s7'
            ]
            
            for col in vp_columns:
                if row.get(col) is not None and row[col] != 0:
                    voting_power[col] = row[col]
            
            update_data = {
                "date": file_date,
                "delegate_id": row['delegate'],
                "voting_power": voting_power,
                "updatedAt": current_time,
                "version": new_version
            }

            # Create update operation
            bulk_operations.append(
                UpdateOne(
                    {
                        "delegate_id": row['delegate'],
                        "date": file_date
                    },
                    {"$set": update_data},
                    upsert=True
                )
            )

        try:
            # Execute bulk operations
            if bulk_operations:
                result = collection.bulk_write(bulk_operations, ordered=False)
                print(f"Processed documents - Inserted: {result.upserted_count}, Modified: {result.modified_count}")

            # Delete old versions
            delete_result = collection.delete_many({
                "version": {"$ne": new_version},
            })
            print(f"Deleted {delete_result.deleted_count} old version documents")
            
            print(f"Data from {filename} saved to MongoDB successfully!")
            return data

        except pymongo.errors.BulkWriteError as bwe:
            # Handle partial success in bulk write
            print(f"Bulk write partial error: {bwe.details}")
            # Cleanup only if no documents were successfully updated
            if bwe.details.get('nInserted', 0) == 0 and bwe.details.get('nModified', 0) == 0:
                collection.delete_many({"version": new_version})
                print(f"Cleaned up version {new_version} due to complete failure")
            raise

    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    finally:
        client.close()

def read_latest_delegate_data(date=None):
    try:
        client = MongoClient(os.getenv('MONGODB_URI'))
        db = client['CPI']
        collection = db['delegate_data']

        # Query filter
        query = {}
        if date:
            query["date"] = date

        # Find the latest version for the given date or overall
        latest_doc = collection.find_one(
            query,
            sort=[("version", pymongo.DESCENDING)]
        )

        if not latest_doc:
            return []

        # Get all documents with the latest version and matching date if specified
        query["version"] = latest_doc["version"]
        latest_data = list(collection.find(query))

        return latest_data

    except Exception as e:
        print(f"Error reading latest data: {e}")
        return None
    finally:
        client.close()

### Calculate HHI and CPI

In [9]:
# Define each influence period with start_date, end_date, and influence percentages
influence_periods = [
    # May 26th, 2022 - January 25th, 2023
    {
        "start_date": "2022-05-26", "end_date": "2023-01-25",
        "influences": {"th_vp": 48.32, "ch_vp_r2": 51.68}
    },
    # January 26th, 2023 - March 30th, 2023
    {
        "start_date": "2023-01-26", "end_date": "2023-03-30",
        "influences": {"th_vp": 41.95, "ch_vp_r2": 44.88, "gc_vp_s3": 13.17}
    },
    # March 31st, 2023 - June 7th, 2023
    {
        "start_date": "2023-03-31", "end_date": "2023-06-07",
        "influences": {"th_vp": 41.95, "ch_vp_r3": 44.88, "gc_vp_s3": 13.17}
    },
    # June 8th, 2023 - January 3rd, 2024
    {
        "start_date": "2023-06-08", "end_date": "2024-01-03",
        "influences": {"th_vp": 41.95, "ch_vp_r3": 44.88, "gc_vp_s4": 13.17}
    },
    # January 4th, 2024 - January 11th, 2024
    {
        "start_date": "2024-01-04", "end_date": "2024-01-11",
        "influences": {
            "th_vp": 32.33, "ch_vp_r3": 34.59, "gc_vp_s5": 10.15,
            "gc_vp_mm_s5": 2.82, "sc_vp_s5": 12.78, "coc_vp_s5": 4.32,
            "dab_vp_s5": 3.01
        }
    },
    # January 12th, 2024 - June 26th, 2024
    {
        "start_date": "2024-01-12", "end_date": "2024-06-26",
        "influences": {
            "th_vp": 32.33, "ch_vp_r4": 34.59, "gc_vp_s5": 10.15,
            "gc_vp_mm_s5": 2.82, "sc_vp_s5": 12.78, "coc_vp_s5": 4.32,
            "dab_vp_s5": 3.01
        }
    },
    # June 27th, 2024 - July 16th, 2024
    {
        "start_date": "2024-06-27", "end_date": "2024-07-16",
        "influences": {
            "th_vp": 32.33, "ch_vp_r4": 34.59, "gc_vp_s6": 10.15,
            "gc_vp_mm_s6": 2.82, "sc_vp_s6": 12.78, "coc_vp_s6": 4.32,
            "dab_vp_s6": 3.01
        }
    },
    # July 17th, 2024 - October 21st, 2024
    {
        "start_date": "2024-07-17", "end_date": "2024-10-21",
        "influences": {
            "th_vp": 32.33, "ch_vp_r5": 34.59, "gc_vp_s6": 10.15,
            "gc_vp_mm_s6": 2.82, "sc_vp_s6": 12.78, "coc_vp_s6": 4.32,
            "dab_vp_s6": 3.01
        }
    },
    # October 22nd, 2024 - January 15th, 2025
    {
        "start_date": "2024-10-22", "end_date": "2025-01-15",
        "influences": {
            "th_vp": 32.33, "ch_vp_r6": 34.59, "gc_vp_s6": 10.15,
            "gc_vp_mm_s6": 2.82, "sc_vp_s6": 12.78, "coc_vp_s6": 4.32,
            "dab_vp_s6": 3.01
        }
    },
    # January 16th, 2025 - June 11th, 2025
    {
        "start_date": "2025-01-16", "end_date": "2025-06-11",
        "influences": {
            "th_vp": 33.73, "ch_vp_r6": 36.08, "gc_vp_s7": 10.59,
            "gc_vp_op_s7": 0.19, "sc_vp_s7": 13.33, "dab_vp_s7": 3.14,
            "mmc_vp_s7": 2.94
        }
    }
]

def calculate_influence(row, influences):
    """Calculates influence based on influence percentages per column."""
    influence_sum = sum(row.get(col, 0) * (val / 100) for col, val in influences.items())
    return influence_sum

def add_influence_column(df, file_date_str):
    """Adds 'influence' column to DataFrame based on file date."""
    file_date = datetime.strptime(file_date_str, "%Y-%m-%d")
    for period in influence_periods:
        start_date = datetime.strptime(period["start_date"], "%Y-%m-%d")
        end_date = datetime.strptime(period["end_date"], "%Y-%m-%d")
        if start_date <= file_date <= end_date:
            df["influence"] = df.apply(calculate_influence, axis=1, influences=period["influences"])
            break
    return df

def calculate_HHI_and_CPI(data, file_date_str):
    """
    Calculate HHI and CPI based on the data and remove the temporary file afterward.
    
    Args:
        data (pd.DataFrame): The processed data.
        file_date_str (str): The date string extracted from the filename.
    
    Returns:
        tuple: HHI and CPI values.
    """
    try:
        # Perform calculations
        data = add_influence_column(data, file_date_str)
        data['th_vp_squared'] = data['th_vp'] ** 2
        data['influence_squared'] = data['influence'] ** 2
        HHI = round(data['th_vp_squared'].sum(), 2)
        CPI = round(data['influence_squared'].sum(), 2)

        print(f"Date: {file_date_str} | HHI: {HHI} | CPI: {CPI}")
        return HHI, CPI
    finally:
        # Construct the file path
        file_path = f"./Data/{file_date_str}.csv"
        
        # Remove the file if it exists
        if os.path.exists(file_path):
            os.remove(file_path)
            print(f"Temporary file removed: {file_path}")

In [10]:
data, filename = fetch_top_5000_delegates()

data = distribute_ACC_Power(data, filename)

data = create_data_sheet(data, filename)

file_date_str = os.path.splitext(filename)[0] 
hhi, cpi = calculate_HHI_and_CPI(data, file_date_str)

C:\Users\yashp\AppData\Local\Temp\ipykernel_31696\2114150361.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_season['address'] = active_season['address'].str.lower()


C:\Users\yashp\AppData\Local\Temp\ipykernel_31696\1704696040.py:20: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  new_version = int(datetime.utcnow().timestamp())


Membership columns added successfully!                                         delegate  voting_power th_vp  \
0     0x2ac5393d1f4be4ef89b45ee2f93d7f20a5cf6d5a  7.024585e+06  None   
1     0x1b686ee8e31c5959d9f5bbd8122a58682788eead  5.738282e+06  None   
2     0x06ad892ce23c136bbda3a821570343a2af3e2914  5.418460e+06  None   
3     0xf11b6a8c3cb8bb7dbc1518a613b10ceb0bbfc06b  5.219295e+06  None   
4     0xeff8d84e0fd304550da242040ccd45bd44ce71f1  5.008845e+06  None   
...                                          ...           ...   ...   
5133  0xe7aa7af667016837733f3ca3809bde04697730ef  0.000000e+00  None   
5134  0x436c7f148cc5dbc90b5531278c87877bdbd83e27  0.000000e+00  None   
5135  0xa8f0048a0d1a04663ca5010d0beac5bcaeea0eef  0.000000e+00  None   
5136  0xe2a0464f8ae3bd34b6cb1e578c1110321f491b72  0.000000e+00  None   
5137  0xd7e7bca98ab9fb25e17ad73429e89a40b55708be  0.000000e+00  None   

      ch_member_r2 ch_vp_r2  ch_member_r3 ch_vp_r3  ch_member_r4 ch_vp_r4  \
0                0 

C:\Users\yashp\AppData\Local\Temp\ipykernel_31696\1704696040.py:194: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.fillna(0, inplace=True)
C:\Users\yashp\AppData\Local\Temp\ipykernel_31696\1704696040.py:197: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_time = datetime.utcnow()


Processed documents - Inserted: 0, Modified: 5138
Deleted 0 old version documents
Data from 2025-03-12.csv saved to MongoDB successfully!
Date: 2025-03-12 | HHI: 220.2 | CPI: 75.56
Temporary file removed: ./Data/2025-03-12.csv
